# Explore how the collect() operation works in PySpark using a dataset with basic RDD operations.

In [1]:
sc


<SparkContext master=local[*] appName=PySparkShell>

# Dataset Overview
Total Records: 50 students Columns: 7 → id, name, age, gender, math, science, english

No missing values

# Demographics
Age: 18 – 25 years (average ≈ 21.5)

Gender: 29 Female, 21 Male

# Academic Performance
Math: Range: 40 – 100 Mean: 68.9 Std. Dev.: 17.6 (high variation)

Science: Range: 44 – 99 Mean: 70.2 Std. Dev.: 14.6 (moderate variation)

English: Range: 42 – 100 Mean: 69.4 Std. Dev.: 18.7 (highest variation)

# Key Insights
Science is the strongest subject on average. English has the most variation in performance. Students perform differently across subjects (not uniform).



In [3]:
data = sc.textFile("students.csv")


In [3]:
header = data.first()
rows = data.filter(lambda line: line != header)


In [4]:
split_rdd = rows.map(lambda line: line.split(","))

In [5]:
print("=== Student Dataset (first 10 rows) ===")
for row in split_rdd.take(10): # you can change 10 → 20, 50 etc.
 print(row)

=== Student Dataset (first 10 rows) ===
['1', 'Alice', '20', 'F', '66', '92', '44']
['2', 'Bob', '20', 'M', '82', '52', '77']
['3', 'Charlie', '22', 'F', '43', '57', '76']
['4', 'David', '19', 'M', '95', '69', '46']
['5', 'Eva', '19', 'F', '62', '44', '96']
['6', 'Frank', '22', 'F', '70', '78', '94']
['7', 'Grace', '24', 'F', '67', '66', '93']
['8', 'Henry', '21', 'F', '53', '82', '60']
['9', 'Ivy', '19', 'M', '64', '52', '46']
['10', 'Jack', '19', 'F', '44', '59', '60']


In [6]:
students_rdd = split_rdd.map(lambda x: (int(x[0]), x[1], int(x[2]), x[3], int(x[4]), int(x[5]), int(x[6])))

In [7]:
avg_marks_rdd = students_rdd.map(lambda x: (x[1], (x[4] + x[5] + x[6]) / 3))


In [8]:
passed_rdd = avg_marks_rdd.filter(lambda x: x[1] >= 75)

In [9]:
sorted_passed_rdd = passed_rdd.sortBy(lambda x: x[1], ascending=False)

In [10]:
results = sorted_passed_rdd.collect()


In [11]:
print("=== Students with Average >= 75 ===")
for student in results:
 print(f"Name: {student[0]}, Avg Marks: {student[1]:.2f}")

=== Students with Average >= 75 ===
Name: Leo, Avg Marks: 88.00
Name: Olivia, Avg Marks: 88.00
Name: Rita, Avg Marks: 86.67
Name: Kathy, Avg Marks: 81.67
Name: George, Avg Marks: 81.67
Name: Frank, Avg Marks: 80.67
Name: Oscar, Avg Marks: 80.00
Name: Uma, Avg Marks: 78.33
Name: Kyle, Avg Marks: 78.33
Name: Matt, Avg Marks: 78.33
Name: Tina, Avg Marks: 76.00
Name: Victor, Avg Marks: 75.67
Name: Grace, Avg Marks: 75.33
Name: Mona, Avg Marks: 75.00
Name: Will, Avg Marks: 75.00


In [12]:
count_passed = passed_rdd.count()
print("\nNumber of students who passed:", count_passed)


Number of students who passed: 15


In [13]:
topper = passed_rdd.reduce(lambda a, b: a if a[1] > b[1] else b)
print("Topper:", topper)


Topper: ('Olivia', 88.0)


In [14]:
print("\nFirst 5 Passed Students (via take):")
print(passed_rdd.take(5))


First 5 Passed Students (via take):
[('Frank', 80.66666666666667), ('Grace', 75.33333333333333), ('Kathy', 81.66666666666667), ('Leo', 88.0), ('Mona', 75.0)]


# Summary
Spark Context setup (sc).

Loading Dataset:

data = sc.textFile("students.csv")
Preprocessing:

Removed header row.
Split lines into fields.
Transformations:

map() → convert rows into structured format.
filter() → apply conditions (e.g., scores/age-based filtering).
flatMap() → expand data where needed.
Actions:

collect() → retrieve all records.
count() → count number of rows.
first(), take(n) → preview sample records.
Aggregations:

reduceByKey() / groupByKey() for subject-wise or student-wise operations.
Possibly word count–style demo for understanding RDD basics.